### This notebook provides the source code and **use cases** for a class from my BTA automation for determing the D&P data blocks and session info.  The first few cells below contain the source code, with **Use Cases** following.

## Sample output:
`{'pre_start_dt': '12/09/2013',
 'start_dt': '09/15/2014',
 'end_dt': '12/28/2015',
 'bt_start_dt': '01/01/2007',
 'bt_end_dt': '11/06/2019',
 'bars_per_session': 6,
 'days_back': 200,
 'session_start_time': 930,
 'session_end_time': 1530,
 'use_daily': True,
 'data_block': 7,
 'bt_duration': 12.85,
 'block_duration': 469}`

### For those of you who have never used a jupyter notebook, there a plenty of tutorials and sample notebooks available - just google.
### Press shift-return in each code block to execute the code 
   

The cell below imports required modules.

In [1]:
import math
from random import randint
from datetime import datetime, timedelta
import pandas as pd
import pprint

The following cell defines DataBlock, the python class that provides functions to calculate the dates for the D&P process and backtesting.

In [2]:
class DataBlock:
    def __init__(self, bt_start_dt=None, bt_end_dt=None, bt_years=10, oos_months=3, num_blocks=10):
        self._bt_start_dt = bt_start_dt
        self._bt_end_dt = bt_end_dt
        self._bt_years = bt_years
        self._oos_months = oos_months
        self._num_blocks = num_blocks

        self.initialize()

    def initialize(self):
        if not self._bt_start_dt:
            if not self._bt_end_dt:
                self._bt_end_dt = (datetime.today() - self._oos_months*timedelta(days=30))
            else:
                self._bt_end_dt = datetime.strptime(self._bt_end_dt, '%m/%d/%Y')
            self._bt_start_dt = self._bt_end_dt - self._bt_years*timedelta(days=365)
        elif not self._bt_end_dt:
            self._bt_start_dt = datetime.strptime(self._bt_start_dt, '%m/%d/%Y')
            if self._oos_months:
                self._bt_end_dt = (datetime.today() - self._oos_months*timedelta(days=30))
            elif self._bt_years:
                self._bt_end_dt = (self._bt_start_dt + self._bt_years*timedelta(days=365))
            if self._bt_end_dt > datetime.today():
                self._bt_end_dt = datetime.today()

    def pick_a_block(self, min_block=1):
        return randint(min_block, self._num_blocks)
    
    def set_dates(self, sess_start, sess_end, bars_back, data_block, entry_tf, use_daily):
        date_format = "%m/%d/%Y"
        bars_per_session = int((self.hhmm2mins(sess_end) - self.hhmm2mins(sess_start)) / entry_tf)
        if use_daily:
            days_back = bars_back
        elif not use_daily:
            days_back = math.ceil(bars_back / bars_per_session)

        seg_size = int((self._bt_end_dt - self._bt_start_dt).days / self._num_blocks)
        start_dt = self._bt_start_dt + pd.DateOffset((data_block - 1) * seg_size)
        pre_start_dt = start_dt - pd.DateOffset(round((days_back / 5) * 7))
        end_dt = start_dt + pd.DateOffset(seg_size)
        return {
            "pre_start_dt": pre_start_dt.strftime(date_format),
            "start_dt":     start_dt.strftime(date_format),
            "end_dt":       end_dt.strftime(date_format),
            "bt_start_dt":  self._bt_start_dt.strftime(date_format),
            "bt_end_dt":    self._bt_end_dt.strftime(date_format),
            "bars_per_session": bars_per_session,
            "days_back": days_back,
            "session_start_time": sess_start,
            "session_end_time": sess_end,
            "use_daily": use_daily,
            "data_block": data_block,
            "bt_duration":   round((self._bt_end_dt - self._bt_start_dt).days/365,2),
            "block_duration":  seg_size,
        }

    #### Private Methods
    def hhmm2mins(self, hhmm):
        i = int(hhmm)
        n_hrs = i // 100
        n_mins = i % 100
        return n_hrs * 60 + n_mins


## **Use Cases:**  The following cells provide examples of usage.  Please note, this code was pulled from by automation framework, and I have a single use case, but implemented several scenaries for requesting dates.  So, not all end-points have been used(tested) much

### Case 1:  Provide backtest start date and the number of months prior to current date to leave of OOS testing.  Use random data block.

In [11]:
dblock = DataBlock(bt_start_dt='1/1/2007',oos_months=6)
dblock_num = dblock.pick_a_block()
print(f"Block number is {dblock_num}")
dblock.set_dates(
    sess_start=930,
    sess_end=1530,
    bars_back=200,
    data_block=dblock_num,
    entry_tf=60,
    use_daily=True
)

Block number is 6


{'pre_start_dt': '08/27/2012',
 'start_dt': '06/03/2013',
 'end_dt': '09/15/2014',
 'bt_start_dt': '01/01/2007',
 'bt_end_dt': '11/06/2019',
 'bars_per_session': 6,
 'days_back': 200,
 'session_start_time': 930,
 'session_end_time': 1530,
 'use_daily': True,
 'data_block': 6,
 'bt_duration': 12.85,
 'block_duration': 469}

### Case 2:  provide backtest End Date and the number of monthts for OOS testing.
  * note: defaults to 10 years of backtesting data

In [5]:
dblock = DataBlock(bt_end_dt='11/1/2020',oos_months=6)
dblock.set_dates(
            sess_start=930,
            sess_end=1530,
            bars_back=200,
            data_block=6,
            entry_tf=60,
            use_daily=True
        )


{'pre_start_dt': '01/27/2015',
 'start_dt': '11/03/2015',
 'end_dt': '11/02/2016',
 'bt_start_dt': '11/04/2010',
 'bt_end_dt': '11/01/2020',
 'bars_per_session': 6,
 'days_back': 200,
 'session_start_time': 930,
 'session_end_time': 1530,
 'use_daily': True,
 'data_block': 6,
 'bt_duration': 10.0,
 'block_duration': 365}

### Case 3:  provide OOS Months and BackTest Years

In [6]:
dblock = DataBlock(oos_months=3,bt_years=11)
dblock.set_dates(
            sess_start=930,
            sess_end=1530,
            bars_back=200,
            data_block=6,
            entry_tf=60,
            use_daily=True
        )


{'pre_start_dt': '10/28/2013',
 'start_dt': '08/04/2014',
 'end_dt': '09/09/2015',
 'bt_start_dt': '02/06/2009',
 'bt_end_dt': '02/04/2020',
 'bars_per_session': 6,
 'days_back': 200,
 'session_start_time': 930,
 'session_end_time': 1530,
 'use_daily': True,
 'data_block': 6,
 'bt_duration': 11.0,
 'block_duration': 401}

### Case 4:  Use pick_a_block to get random block number >=6, system defaults (10 years backtesting, 6 months OOS)

In [7]:
dblock = DataBlock(bt_start_dt='1/1/2007',oos_months=6)
dblock_num = dblock.pick_a_block(6)
print(f"Block number is {dblock_num}")
dblock.set_dates(
            sess_start=930,
            sess_end=1530,
            bars_back=200,
            data_block=dblock_num,
            entry_tf=60,
            use_daily=True
        )


Block number is 6


{'pre_start_dt': '08/27/2012',
 'start_dt': '06/03/2013',
 'end_dt': '09/15/2014',
 'bt_start_dt': '01/01/2007',
 'bt_end_dt': '11/06/2019',
 'bars_per_session': 6,
 'days_back': 200,
 'session_start_time': 930,
 'session_end_time': 1530,
 'use_daily': True,
 'data_block': 6,
 'bt_duration': 12.85,
 'block_duration': 469}

### Case 5:  Use 6 data blocks instead of the standard 10

In [8]:
dblock = DataBlock(num_blocks=6)
dblock_num = dblock.pick_a_block(3)
print(f"data block number = {dblock_num}")
dblock.set_dates(
            sess_start=930,
            sess_end=1530,
            bars_back=200,
            data_block=1,
            entry_tf=60,
            use_daily=True
        )


data block number = 4


{'pre_start_dt': '05/02/2009',
 'start_dt': '02/06/2010',
 'end_dt': '10/07/2011',
 'bt_start_dt': '02/06/2010',
 'bt_end_dt': '02/04/2020',
 'bars_per_session': 6,
 'days_back': 200,
 'session_start_time': 930,
 'session_end_time': 1530,
 'use_daily': True,
 'data_block': 1,
 'bt_duration': 10.0,
 'block_duration': 608}